In [270]:
import pandas as pd
import numpy as np
from pandas import DataFrame

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from tqdm.notebook import tqdm

apidata = pd.read_csv("./github/Dataset/210503_enrichedBookData.csv", sep="|", error_bad_lines=False)
book = pd.read_csv("./items.csv", sep="|")

b'Skipping line 472: expected 6 fields, saw 12\n'


In [271]:
book

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]"
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]"
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]"
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]"
...,...,...,...,...,...,...
78025,37678,Timeless Fairy Tales,"Brothers Grimm, Marie-Catherine Baroness D'Aulnoy",MEDIAMORPHOSIS,YFA,[YFJ]
78026,68688,Demon Games,A. Witt Timothy a. Witt,iUniverse,FMB,[]
78027,57291,Lori and the Lion's Den,A. M. Glass,Xlibris,YFU,[]
78028,78130,The Everywhere Armchair,Ersila Bee,ELOQUENT BOOKS,YFC,"[YFG,YFH]"


In [272]:
apidata.sort_values('titleQueried')
apidata.head()

,titleQueried,titleFound,author,language,description,isbn;;;;;;;
0,Princess Poppy: The Big Mix Up,The Big Mix-up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9780552555968;;;;;;;
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9781409048787;;;;;;;
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9781409048817;;;;;;;
3,Princess Poppy: The Big Mix Up,Princess Poppy: The Fashion Princess,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9781407098197;;;;;;;
4,Princess Poppy: The Big Mix Up,Princess Poppy: The Rescue Princess,Janey Louise Jones,en,A brand-new story in the best-selling Princess...,9781407098296;;;;;;;


In [273]:
apidata.isnull().sum()

titleQueried        0
titleFound      11270
author          20614
language        11649
description     41485
isbn;;;;;;;     12912
dtype: int64

In [274]:
apidata.shape

(81459, 6)

In [275]:
# remove row with no author
processedApi = apidata[apidata['author'].notna()]

In [276]:
processedApi.isnull().sum()

titleQueried        0
titleFound          0
author              0
language           32
description     21546
isbn;;;;;;;      1282
dtype: int64

In [277]:
# remove special characters in author and isbn
processedApi.columns = processedApi.columns.str.replace(';', '')
processedApi["isbn"] = processedApi["isbn"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
processedApi["author"] = processedApi["author"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
processedApi

<ipython-input-277-ff03607e091a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processedApi["isbn"] = processedApi["isbn"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
<ipython-input-277-ff03607e091a>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  processedApi["author"] = processedApi["author"].str.replace(pat=r'[^\w]', repl=r'', regex=True)


,titleQueried,titleFound,author,language,description,isbn
0,Princess Poppy: The Big Mix Up,The Big Mix-up,JaneyLouiseJones,en,A brand-new story in the best-selling Princess...,9780552555968
1,Princess Poppy: The Big Mix Up,Princess Poppy: The Big Mix Up,JaneyLouiseJones,en,A brand-new story in the best-selling Princess...,9781409048787
2,Princess Poppy: The Big Mix Up,Princess Poppy: The Haunted Holiday,JaneyLouiseJones,en,A brand-new story in the best-selling Princess...,9781409048817
3,Princess Poppy: The Big Mix Up,Princess Poppy: The Fashion Princess,JaneyLouiseJones,en,A brand-new story in the best-selling Princess...,9781407098197
4,Princess Poppy: The Big Mix Up,Princess Poppy: The Rescue Princess,JaneyLouiseJones,en,A brand-new story in the best-selling Princess...,9781407098296
...,...,...,...,...,...,...
81454,Diablo: Der dunkle Pfad,Diablo,RichardAKnaak,de,Getrieben von schrecklichen Albträumen findet ...,9783736798960
81455,Diablo: Der dunkle Pfad,Kosmospfade,MariaSusanaCipolletti,de,Die vorliegende Studie behandelt die vielfalti...,9783896658227
81456,Diablo: Der dunkle Pfad,Diablo,RichardAKnaak,en,NaN,9783833212673
81457,Diablo: Der dunkle Pfad,Die Welt von Cyberpunk 2077,MarcinBatylda,de,Man schreibt das Jahr 2077. Die Welt ist gespi...,9783736798915


In [278]:
# make a dataframe w/ books in common between original data and apidata
mask = processedApi['titleQueried'].isin(book['title'])
booksInCommon,processedApi_left = processedApi[mask],processedApi[~mask]

In [279]:
booksInCommon = booksInCommon.sort_values('titleQueried')
booksInCommon['key:title-author'] = ((booksInCommon['titleQueried'] + ' '+ booksInCommon['author']).astype(str)).str.lower()
booksInCommon = booksInCommon.drop_duplicates('key:title-author', keep='first')
# booksInCommon.to_excel('check2.xlsx')

In [280]:
book['key:title-author'] = ((book['title'] + ' '+ book['author']).astype(str)).str.lower()
book = book.sort_values('title')
book

,itemID,title,author,publisher,main topic,subtopics,key:title-author
34478,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],tripion minaccia dallo spazio stefano grimaldi
66492,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],the ultimate vehicle colouring book for kids ...
31055,64846,!!,NaN,AUTHORHOUSE,FM,[],nan
74481,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,DARBY CREEK,YF,[5HKA],"#4 happy birthday, mallory! laurie friedman"
2806,7780,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Frech Verlag GmbH,WF,"[5AF,Y]","#basteln for future susanne pypke, naturschutz..."
...,...,...,...,...,...,...,...
61016,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires",MOZAIKA LLC,FMX,[],übernatürliche irreführung (sasha urban serie ...
56197,35275,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,Books on Demand,FD,"[FD,FM]",überschneidung zweier untersschiedlicher welte...
56191,50552,Übersinnlicher Retter,Tina Folsom,Duboce Park Press,FRT,[],übersinnlicher retter tina folsom
20130,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,"Sunshine Handlettering, Handlettering Heft In ...",INDEPENDENTLY PUBLISHED,YNV,[],übungsheft handlettering: übungsbuch für hand ...


In [281]:
# check similarity
dftemp = DataFrame (range(0),columns=list(booksInCommon.columns))
threshold = 80
threshold2 = 70
j_end = 0

for i in tqdm(range(len(book))):    
    for j in range(j_end,len(booksInCommon)):
        # if titles from two data are the same
        if book['title'].iloc[i] == booksInCommon.get('titleQueried').iloc[j]:
            j_end += 1
            # if similarity between key:title-author > threshold
            if (fuzz.token_sort_ratio(booksInCommon.get('key:title-author').iloc[j], book.get('key:title-author').iloc[i])) > threshold:
                # if similarity between titleQueried and titleFound  > threshold2, then save it as a dataframe
                if (fuzz.token_sort_ratio(booksInCommon.get('titleQueried').iloc[j], booksInCommon.get('titleFound').iloc[j])) > threshold2:
                    dftemp = dftemp.append(booksInCommon.iloc[j])     
        else:
            j_end = j
            break;

In [282]:
dftemp

,titleQueried,titleFound,author,language,description,isbn,key:title-author
4135,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9783772449680,#wir bleiben zuhause kindermitmachbuch frechve...
29988,1000 Gefahren in der versunkenen Stadt,1000 Gefahren in der versunkenen Stadt,FabianLenk,de,NaN,9783473525904,1000 gefahren in der versunkenen stadt fabianlenk
57742,107 Schläge pro Minute,107 Schläge pro Minute,ZsókaSchwab,de,Die achtzehnjährige Romy leidet an einer unerk...,9783967140569,107 schläge pro minute zsókaschwab
7396,111 x Papierfalten - Ostern,111 x Papierfalten - Ostern,ThadePrecht,de,NaN,9783745903898,111 x papierfalten - ostern thadeprecht
17306,12 things to do before you crash and burn,12 things to do before you crash and burn,JamesProimos,de,NaN,9783407745255,12 things to do before you crash and burn jame...
...,...,...,...,...,...,...,...
15298,black stories junior - spooky stories,black stories junior - spooky stories,CorinnaHarder,en,NaN,9783964550088,black stories junior - spooky stories corinnah...
76106,grydscaen,Grydscaen,NatsuyaUesugi,en,Lino Dejarre is the son of the Viceroy of the ...,9781479700172,grydscaen natsuyauesugi
60241,moses. - Pocket Quiz junior - Schlaumeier,Pocket Quiz junior Tiere,JürgenWinzer,en,NaN,9783964551030,moses. - pocket quiz junior - schlaumeier jürg...
32561,"»Super«, schreit der Frieder, und die Oma kich...","»Super«, schreit der Frieder, und die Oma kich...",GudrunMebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9783733650919,"»super«, schreit der frieder, und die oma kich..."


In [283]:
# check whether they are from collection, the same book, or different book
n = 8

dftemp2 = dftemp.copy()
dftemp2['titleQueried'] = (dftemp2['titleQueried']).astype(str).str.lower()
dftemp2['titleFound'] =(dftemp2['titleFound']).astype(str).str.lower()
dftemp2["titleQueried"] = dftemp2["titleQueried"].str.replace(pat=r'[^\w]', repl=r'', regex=True)
dftemp2["titleFound"] = dftemp2["titleFound"].str.replace(pat=r'[^\w]', repl=r'', regex=True)

diffList = []
sameList = []
collectionList = []

for i in range(len(dftemp2)):
    # if title is not the same --> need to further check
    if dftemp2.get('titleQueried').iloc[i] != dftemp2.get('titleFound').iloc[i]:
        ## if titleQueried and titleFound similarity < 97 --> need to further check 
        if fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) < 97:
            ### if firt n-letter similarity > 80 --> they are from a collection
            if (fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i][:n], dftemp2.get('titleFound').iloc[i][:n]) > 80):
                collectionList.append(dftemp.iloc[i])
            elif fuzz.token_sort_ratio(dftemp2.get('titleQueried').iloc[i], dftemp2.get('titleFound').iloc[i]) > 80:
                collectionList.append(dftemp.iloc[i])
            else:
                diffList.append(dftemp.iloc[i])
        ## if titleQueried and titleFound similarity > 97, they are the same book 
        else:
            sameList.append(dftemp.iloc[i])
    # if titme is the same, they are the same book
    else:
        sameList.append(dftemp.iloc[i])
            
collectionDf = pd.DataFrame(collectionList)
diffDf = pd.DataFrame(diffList)
sameDf = pd.DataFrame(sameList)

In [284]:
#fuzz.token_sort_ratio('Alex Rider 2: Stormbreaker', 'Alex Rider 1: Stormbreaker')

### collection: collection(1) same book(2)

In [285]:
# collectionDf, diffDf, sameDf
# diffDf.to_excel('diffDf.xlsx')
diffDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author
10706,20 Erzählungen und Kurzgeschichten,Neue Kurzgeschichten und Erzählungen,TorstenLöschmann,de,NaN,9783403505655,20 erzählungen und kurzgeschichten torstenlösc...
26421,A Clash of Kings: The Illustrated Edition,Nightflyers: The Illustrated Edition,GeorgeRRMartin,en,A gorgeous illustrated edition of a classic no...,9780525619697,a clash of kings: the illustrated edition geor...
64505,Adventures of Huckleberry Finn,The Adventures of Huckleberry Finn (Tom Sawyer...,MarkTwain,en,NaN,,adventures of huckleberry finn marktwain
68963,Bastel-Kids - Papierfalten im Quadrat,Papierfalten im Quadrat: Lieblingstiere - Bast...,ThadePrecht,de,NaN,9783745903744,bastel-kids - papierfalten im quadrat thadeprecht
12560,Benjamin Blümchen: Meine Kindergartenfreunde,Alle meine Kindergartenfreunde,AnnetRudolph,de,Damit vergisst man seineSpielkameraden aus dem...,9783480302352,benjamin blümchen: meine kindergartenfreunde a...
...,...,...,...,...,...,...,...
6021,Wütender Sturm (Die Farben des Blutes 4),Die Farben des Blutes,VictoriaAveyard,en,NaN,,wütender sturm (die farben des blutes 4) victo...
60730,Young Bond: Hurricane Gold,Hurricane Gold,CharlieHigson,en,As the sun blazes over the Caribbean island of...,9780141343402,young bond: hurricane gold charliehigson
60731,Young Bond: Hurricane Gold,Hurricane Gold,CharlesHigson,en,"Future spy James Bond, a teenager in the 1930s...",9781442087569,young bond: hurricane gold charleshigson
8737,Zorn der Engel,Rückkehr der Engel,MarahWoolf,en,NaN,9783492706018,zorn der engel marahwoolf


### need to go over diffDF/collectionDF mannually --> add to  sameDf / collectionDf if necessary

In [286]:
collectionDf['collection'] = 1
collectionDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author,collection
4657,"17, Das dritte Buch der Erinnerung",17 - Das Vierte Buch Der Erinnerung,RoseSnow,de,"Was w�rdest du tun, wenn du pl�tzlich in fremd...",9781545359440,"17, das dritte buch der erinnerung rosesnow",1
2785,"17, Das erste Buch der Erinnerung",17 - Das Vierte Buch Der Erinnerung,RoseSnow,de,"Was w�rdest du tun, wenn du pl�tzlich in fremd...",9781545359440,"17, das erste buch der erinnerung rosesnow",1
14675,19 - Das zweite Buch der magischen Angst,19 - Das erste Buch der magischen Angst,RoseSnow,de,NaN,9783473585922,19 - das zweite buch der magischen angst rosesnow,1
10709,20 Erzählungen und Kurzgeschichten,"Erzählungen, Kurzgeschichten, Märchen",PeterWimmer,de,INHALT: Achmed Omara Ali - Der Prinz vom Nil /...,9783741851940,20 erzählungen und kurzgeschichten peterwimmer,1
32751,39-Storey Treehouse,The 39-storey Treehouse,AndyGriffiths,en,Join Andy and Terry in their astonishing 39-st...,9781742612379,39-storey treehouse andygriffiths,1
...,...,...,...,...,...,...,...,...
36429,Wings of Fire 5,Wings of Fire 6,TuiTSutherland,en,NaN,9783948638337,wings of fire 5 tuitsutherland,1
35763,Wächter der Runen (Band 1),Wächter der Runen,JKBloom,en,NaN,9783906829951,wächter der runen (band 1) jkbloom,1
37824,Wächter der Runen (Band 2),Wächter der Runen,JKBloom,en,NaN,9783906829951,wächter der runen (band 2) jkbloom,1
66636,Zeichnen für Anfänger Schritt für Schritt (Ras...,Zeichnen für Anfänger Schritt für Schritt (Ras...,JamesManning,de,"Dieses Buch bringt Kindern bei, wie man Comic-...",9781839803093,zeichnen für anfänger schritt für schritt (ras...,1


In [287]:
sameDf['collection'] = 2
sameDf

,titleQueried,titleFound,author,language,description,isbn,key:title-author,collection
4135,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9783772449680,#wir bleiben zuhause kindermitmachbuch frechve...,2
29988,1000 Gefahren in der versunkenen Stadt,1000 Gefahren in der versunkenen Stadt,FabianLenk,de,NaN,9783473525904,1000 gefahren in der versunkenen stadt fabianlenk,2
57742,107 Schläge pro Minute,107 Schläge pro Minute,ZsókaSchwab,de,Die achtzehnjährige Romy leidet an einer unerk...,9783967140569,107 schläge pro minute zsókaschwab,2
7396,111 x Papierfalten - Ostern,111 x Papierfalten - Ostern,ThadePrecht,de,NaN,9783745903898,111 x papierfalten - ostern thadeprecht,2
17306,12 things to do before you crash and burn,12 things to do before you crash and burn,JamesProimos,de,NaN,9783407745255,12 things to do before you crash and burn jame...,2
...,...,...,...,...,...,...,...,...
15308,black stories Junior - rainbow stories,black stories Junior - rainbow stories,CorinnaHarder,en,NaN,9783964550071,black stories junior - rainbow stories corinna...,2
15298,black stories junior - spooky stories,black stories junior - spooky stories,CorinnaHarder,en,NaN,9783964550088,black stories junior - spooky stories corinnah...,2
76106,grydscaen,Grydscaen,NatsuyaUesugi,en,Lino Dejarre is the son of the Viceroy of the ...,9781479700172,grydscaen natsuyauesugi,2
32561,"»Super«, schreit der Frieder, und die Oma kich...","»Super«, schreit der Frieder, und die Oma kich...",GudrunMebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9783733650919,"»super«, schreit der frieder, und die oma kich...",2


In [288]:
# mergedDf (sameDf + collectionDf)
merged = pd.concat([sameDf, collectionDf])
merged.sort_values('titleQueried')
merged.rename(columns = {'titleQueried':'title'}, inplace=True)
merged

,title,titleFound,author,language,description,isbn,key:title-author,collection
4135,#Wir bleiben zuhause Kindermitmachbuch,#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9783772449680,#wir bleiben zuhause kindermitmachbuch frechve...,2
29988,1000 Gefahren in der versunkenen Stadt,1000 Gefahren in der versunkenen Stadt,FabianLenk,de,NaN,9783473525904,1000 gefahren in der versunkenen stadt fabianlenk,2
57742,107 Schläge pro Minute,107 Schläge pro Minute,ZsókaSchwab,de,Die achtzehnjährige Romy leidet an einer unerk...,9783967140569,107 schläge pro minute zsókaschwab,2
7396,111 x Papierfalten - Ostern,111 x Papierfalten - Ostern,ThadePrecht,de,NaN,9783745903898,111 x papierfalten - ostern thadeprecht,2
17306,12 things to do before you crash and burn,12 things to do before you crash and burn,JamesProimos,de,NaN,9783407745255,12 things to do before you crash and burn jame...,2
...,...,...,...,...,...,...,...,...
36429,Wings of Fire 5,Wings of Fire 6,TuiTSutherland,en,NaN,9783948638337,wings of fire 5 tuitsutherland,1
35763,Wächter der Runen (Band 1),Wächter der Runen,JKBloom,en,NaN,9783906829951,wächter der runen (band 1) jkbloom,1
37824,Wächter der Runen (Band 2),Wächter der Runen,JKBloom,en,NaN,9783906829951,wächter der runen (band 2) jkbloom,1
66636,Zeichnen für Anfänger Schritt für Schritt (Ras...,Zeichnen für Anfänger Schritt für Schritt (Ras...,JamesManning,de,"Dieses Buch bringt Kindern bei, wie man Comic-...",9781839803093,zeichnen für anfänger schritt für schritt (ras...,1


In [289]:
# add api data to items.csv data
addedData = pd.merge(book, merged, how="outer", on="title")
addedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
addedData

,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,26966,TRIPION minaccia dallo spazio,Stefano Grimaldi,Lulu.com,FL,[],NaN,NaN,NaN,NaN,NaN,NaN
1,24125,The Ultimate Vehicle Colouring Book for Kids,Chetna .,Westland Publications Limited,YBG,[],NaN,NaN,NaN,NaN,NaN,NaN
2,64846,!!,NaN,AUTHORHOUSE,FM,[],NaN,NaN,NaN,NaN,NaN,NaN
3,64745,"#4 Happy Birthday, Mallory!",Laurie Friedman,DARBY CREEK,YF,[5HKA],NaN,NaN,NaN,NaN,NaN,NaN
4,7780,#Basteln for Future,"Susanne Pypke, Naturschutzjugend NAJU",Frech Verlag GmbH,WF,"[5AF,Y]",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
78154,21473,Übernatürliche Irreführung (Sasha Urban Serie 5),"Dima Zales, Anna Zaires",MOZAIKA LLC,FMX,[],NaN,NaN,NaN,NaN,NaN,NaN
78155,35275,Überschneidung zweier untersschiedlicher Welten,WErner Röschl,Books on Demand,FD,"[FD,FM]",NaN,NaN,NaN,NaN,NaN,NaN
78156,50552,Übersinnlicher Retter,Tina Folsom,Duboce Park Press,FRT,[],NaN,NaN,NaN,NaN,NaN,NaN
78157,5665,Übungsheft Handlettering: Übungsbuch Für Hand ...,"Sunshine Handlettering, Handlettering Heft In ...",INDEPENDENTLY PUBLISHED,YNV,[],NaN,NaN,NaN,NaN,NaN,NaN


In [290]:
joinedData = pd.merge(book, merged, how="inner", on="title")
joinedData.drop(['key:title-author_x', 'key:title-author_y'], axis=1, inplace=True)
joinedData

,itemID,title,author_x,publisher,main topic,subtopics,titleFound,author_y,language,description,isbn,collection
0,37807,#Wir bleiben zuhause Kindermitmachbuch,frechverlag,Frech Verlag GmbH,YBGC,"[5AF,5JA,5JB,YBGC,YBL,YZG]",#Wir bleiben zuhause Kindermitmachbuch,Frechverlag,de,NaN,9783772449680,2
1,31991,1000 Gefahren in der versunkenen Stadt,Fabian Lenk,Ravensburger Verlag,YFC,[5AL],1000 Gefahren in der versunkenen Stadt,FabianLenk,de,NaN,9783473525904,2
2,26243,107 Schläge pro Minute,Zsóka Schwab,Zeilenfluss,FXB,"[FMW,FMX]",107 Schläge pro Minute,ZsókaSchwab,de,Die achtzehnjährige Romy leidet an einer unerk...,9783967140569,2
3,30790,111 x Papierfalten - Ostern,Thade Precht,Edition Michael Fischer,WFTM,"[5AF,5HPF,YBG,YBL]",111 x Papierfalten - Ostern,ThadePrecht,de,NaN,9783745903898,2
4,49966,12 things to do before you crash and burn,James Proimos,"Beltz GmbH, Julius",YF,[5AQ],12 things to do before you crash and burn,JamesProimos,de,NaN,9783407745255,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2410,65052,grydscaen,Natsuya Uesugi,Xlibris,FL,[],Grydscaen,NatsuyaUesugi,en,Lino Dejarre is the son of the Viceroy of the ...,9781479700172,2
2411,44283,grydscaen,Natsuya Uesugi,Xlibris,FL,[],Grydscaen,NatsuyaUesugi,en,Lino Dejarre is the son of the Viceroy of the ...,9781479700172,2
2412,34490,grydscaen,Natsuya Uesugi,Xlibris,FL,[],Grydscaen,NatsuyaUesugi,en,Lino Dejarre is the son of the Viceroy of the ...,9781479700172,2
2413,56883,"»Super«, schreit der Frieder, und die Oma kich...",Gudrun Mebs,FISCHER KJB,YFM,"[5AG,YFN,YFQ]","»Super«, schreit der Frieder, und die Oma kich...",GudrunMebs,de,Der fünfte Band rund um Frieder und Oma und Fr...,9783733650919,2
